### Welcome to the first exercise on E-ID in this E-ID Workshop
Before we start, make sure you have everything installed by following one of the outlined methods in the ReadMe file. 

### Purpose of this exercise
This jupyter notebook explains the following:
1. Basic E-ID example using RSA cryptographic scheme
2. Selective Disclosure using RSA and its downsides
3. What is unlinkability in the E-ID world
4. The previous points should explain to a degree the motivation behind the need for a more specialized cryptographic scheme

In [ ]:
const date = new Date("1993-08-01T00:00:00")
date.getTime()

In [ ]:
// We start by creating a typical E-ID object that we will use through out this exercise
const ID_DATA = {
    name: "Jack Sparrow",
    timeOfBirth: date.getTime(),
    profession: "IT Manager"
}

## Definition: Verifiable Credentials
A verifiable credential, in its simplest form, is a signed string of data. An issuer will issue a credential by signing a specific string of data then sharing that string of data along with a cryptographic signature that can prove that this string was authorized/ issued by this specific issuer.

In [ ]:
import * as crypto from 'crypto';

### Issuer

In [ ]:
/* 
 For an issuer to be able to start issuing Verifiable Credentials, it first needs
 to have its own cryptographic key pair. 
 Issuers will sign the data using their private key. 
*/
const { publicKey, privateKey } = crypto.generateKeyPairSync('rsa', {
    modulusLength: 4096,
    publicKeyEncoding: {
      type: 'spki',   
    
      format: 'pem'
    },
    privateKeyEncoding: {
      type: 'pkcs8',
      format: 'pem',
    }
});

In [ ]:
// An issuer will convert the data object into a string
const message = JSON.stringify(ID_DATA)

// then create a signature for that data
const signer = crypto.createSign('SHA256');
signer.update(message);
const signature = signer.sign(privateKey, 'base64')

console.log(message);
console.log("----------------------------")
console.log(signature);

### Issuer →  Holder

After the signature is created, the data along with the signature is transfered to the holder

In this case, the holder can only do one thing with this data which is to share the whole data string along with the signature

### Holder →  Verifier

Once the data is sent from holder to verifier, the verifier can verify that information as follows:

In [ ]:
// verifier now recieves "message" and "signature" data
// The verifier should also have access to the publicKey of the issuer
// through a Public Key Infrastructure (PKI)

let verifier = crypto.createVerify('SHA256');
verifier.update(message)
console.log(verifier.verify(publicKey, signature, 'base64'))

In [ ]:
// What happens if the data is not accurate or true?

verifier = crypto.createVerify('SHA256');
verifier.update("Another random message!")
console.log(verifier.verify(publicKey, signature, 'base64'))

---

## Selective disclosure

What if the holder of the credential wants to only share his name and profession but not his timeOfBirth?

The current implementation wouldn't allow for that, so we will need to change it to some extent.

In [ ]:
// One way to implement selective disclosure is to hash every value, and the 
// holder of the credential would share only the values he intends to. 

// issuer will hash the values before signing the data
function hashValue(value: string): string {
  const hash = crypto.createHash('sha256');
  hash.update(value);
  return hash.digest('hex');
}
const ID_DATA_HASHED = {
    name: hashValue(ID_DATA['name']),
    timeOfBirth: hashValue(ID_DATA['timeOfBirth'].toString()),
    profession: hashValue(ID_DATA['profession'])
}

// An issuer will convert the data object into a string
const message = JSON.stringify(ID_DATA_HASHED);

// then create a signature for that data
const signer = crypto.createSign('SHA256');
signer.update(message);

const signature = signer.sign(privateKey, 'base64');
console.log(message);
console.log("----------------------------");
console.log(signature);

In [ ]:
// Now, similarly a verifier can use the same mechanism to verify that a given data is accurate

let verifier = crypto.createVerify('SHA256');
verifier.update(message)
console.log(verifier.verify(publicKey, signature, 'base64'))

In [ ]:
// But the content of this message is not useful by itself, so the holder needs to attach the unhashed data

// For example, the holder can share the value for the name and and profession
const HOLDER_SHARED_DATA = {
    name: "Jack Sparrow",
    profession: "IT Manager"
}

In [ ]:
// Now, a verifier can use this information to ensure that the unhashed data is correct.
// By rebuilding the hashed data object

// Step 0. verifier's own hash function
function hashValue(value: string): string {
  const hash = crypto.createHash('sha256');
  hash.update(value);
  return hash.digest('hex');
}

// Step 1. Retrieve data object from JSON data
const RETRIEVED_DATA = JSON.parse(message);

// step 2. Make sure the reconstructed object is the same as the original hashed data
for (const key in HOLDER_SHARED_DATA) {
    if (hashValue(HOLDER_SHARED_DATA[key]) != RETRIEVED_DATA[key]) {
        console.log(`Reconstructed data in key ${key} is not the same as the hashed counterpart`);
    }
}

console.log(RETRIEVED_DATA)

// Since we've already verified that the hashed message is valid in the previous code cell
// and now we verified that the hashed values are equal to the revealed values, then
// we conclude that we trust these revealed data.

# Discussion

Question: Ask about the downsides of using hashed values in E-ID?

---

## Unlinkability

Unlinkability means that a verifier who interacts with a credential holder in an attempt to get access to some verifiable credentials, that verifiable should be unable to link different transactions made by the same holder using the same credential or different credentials belonging to the same holder.

In other words, the different verifications are **unlinkable** with respect to the verifier.

It's also important that the issuer of the credential doesn't have access to any information about when or how the issued credential is used.

### Is this cryptographic scheme unlinkable?

No! because verifiers can store the signature which will always stay the same. This way, verifiers can compare signatures, and deduce that it's the same credential by the same user.  
Also the hashed values will always stay the same.

Another reason is that verifiers always have access to all the data in the hashed form, so even though they don't know the hashed value, they can still compare hashed values together, and deduce that result. For fields like age, it's also easy to guess the hidden value. 

In [ ]:
// data in hashed form. hashing an timeOfBirth of 61 will always be the same.
RETRIEVED_DATA

## Coding Exercise: 
Right now, the timeOfBirth can be guessed by looping over all possible dates to figure out someone's exact date of birth. 

### Task
Reimplement the communication between the holder and verifier modifying the hash function in a way that doesn't make it easy to guess someone's date of birth.

!Ask for a hint!